# 06장 어댑터

* 이름: 홍길동
* 학번: 2024123456
* 조: 1

## 공통 코드

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import io
import os
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from scipy import interpolate
from numpy import nan, inf

from IPython.display import Image

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def tangent_line_end(f, x):
    h = 1e-4
    d = (f(x) - f(x-h)) / (h)
    return lambda t: d*t - d*x + f(x)

def tangent_line_and_gradient(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return (lambda t: d*t - d*x + f(x)), d

def tangent_line_and_gradient_end(f, x):
    h = 1e-4
    d = (f(x) - f(x-h)) / (h)
    return (lambda t: d*t - d*x + f(x)), d

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if not os.path.exists(file_name):
        print("{} doesn't exist.".format(file_name))
        return None

    if end == -1:
        end = None
    else:
        end += 1

    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []

            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]

            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if not os.path.exists(file_name):
        print("{} doesn't exist.".format(file_name))
        return None

    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def print_value_to_string(label, value):
    output = io.StringIO()
    print('%s = ' % label, end='', file=output)
    print('{:11.3f}'.format(value), file=output)
    captured = output.getvalue()
    return captured

def draw_plot(xs, ys, label, style_idx, color_idx=-1, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-',
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if color_idx < 0:
        color_idx = style_idx

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[color_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[color_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []


def calculate_square_wave_frequency(times, vins):
    # 전압 값의 중간값을 임계값으로 설정
    threshold = (np.max(vins) + np.min(vins)) / 2

    # 상승 에지 감지
    rising_edges = np.where((vins[:-1] < threshold) & (vins[1:] >= threshold))[0]

    if len(rising_edges) < 2:
        return None  # 주기를 계산할 수 없음

    # 주기 계산 (연속된 상승 에지 사이의 평균 시간)
    periods = times[rising_edges[1:]] - times[rising_edges[:-1]]
    average_period = np.mean(periods)

    # 주파수 계산 (주기의 역수)
    frequency = 1 / average_period

    return frequency

def display_image(file_path, width=None, height=None):
    if os.path.exists(file_path):
        display(Image(file_path, width=width, height=height))
    else:
        print("{} doesn't exist.".format(file_path))

def find_first_peak_index(xs, ys, margine=10):
    xs_len = len(xs)
    old_grad = None
    pi = -1
    m = margine
    for i in range(xs_len - 1):
        grad = (ys[i+1] - ys[i]) / (xs[i+1] - xs[i])
        if grad > 0:
            pi = i
            m -= 1
            if m == 0:
                old_grad = grad
                break
        else:
            m = margine
    if old_grad != None:
        m = margine
        for i in range(pi, xs_len - 1):
            grad = (ys[i+1] - ys[i]) / (xs[i+1] - xs[i])
            if grad < 0:
                pi = i
                m -= 1
                if m == 0:
                    pi -= margine
                    break
            else:
                m = margine
    return pi


*******************************************************************************
## 실험01 반파정류 회로

### 실험01-01 회로의 구성

#### 실험 방법

* 다이오드: 1N4004
* RL: 1 kOhm

In [ ]:
display_image("./images/ch06_ep01_01_circuit.png", width=600)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep01_01_result_circuit.jpg", width=1000)

### 실험01-02 파형발생기로 실험

#### 실험 방법

* Vi: 정현파, 주파수 60 Hz, 크기 8 Vpp

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep01_02_result.png", width=600)
display_image("./data/ch06_ep01_02_result_xy.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep01_02_result.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 10, 2)
    vouts = savgol_filter(vouts, 10, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.xlim((0, 0.05))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT)', 1)
    plt.show()

    vi_pi = find_first_peak_index(times, vins)
    vo_pi = find_first_peak_index(times, vouts)
    V_diff_V = vins[vi_pi] - vouts[vo_pi]
    print_value("V_diff_V", V_diff_V)

    # 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
    plt.xlabel('V_IN_V')
    plt.ylabel('V_OUT_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    m = int(len(vins) / 10)
    draw_plot(vins[m:-m], vouts[m:-m], "XY", 0)
    plt.show()


* 고찰:
    + Vi가 ?? 보다 클 때만 ??가 Vo로 출력된다.
    + 결과적으로 ??정류가 이루어진다.

### 실험01-03 트랜스포머로 실험

#### 실험 방법

* Vi: 트랜스포머 (AC 220 V -> 약 25 Vpp)

In [ ]:
display_image("./images/ch06_ep01_03_circuit.png", width=600)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep01_03_result_circuit.jpg", width=1000)

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep01_03_result.png", width=600)
display_image("./data/ch06_ep01_03_result_xy.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep01_03_result.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 10, 2)
    vouts = savgol_filter(vouts, 10, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.xlim((0, 0.05))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT)', 1)
    plt.show()

    vi_pi = find_first_peak_index(times, vins)
    vo_pi = find_first_peak_index(times, vouts)
    V_diff_V = vins[vi_pi] - vouts[vo_pi]
    print_value("V_diff_V", V_diff_V)

    # 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
    plt.xlabel('V_IN_V')
    plt.ylabel('V_OUT_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    m = int(len(vins) / 10)
    draw_plot(vins[m:-m], vouts[m:-m], "XY", 0)
    plt.show()


* 고찰:
    + 실험01-02 유사하게 ??정류가 이루어진다.

*******************************************************************************
## 실험02 전파정류 회로

### 실험02-01 회로의 구성

#### 실험 방법

* 다이오드: 1N4004
* RL: 1 kOhm

In [ ]:
display_image("./images/ch06_ep02_01_circuit.png", width=600)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep02_01_result_circuit.jpg", width=1000)

### 실험02-02 파형발생기로 실험

#### 실험 방법

* Vi: 정현파, 주파수 60 Hz, 크기 12 Vpp
* Vi와 Vo를 각각 측정한다. (동시에 측정하면 Vi-가 GND와 연결되어 정상동작하지 않음)

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep02_02_result_vi.png", width=600)
display_image("./data/ch06_ep02_02_result_vo.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data_vi = get_oscilloscpoe_result_tektronix('./data/ch06_ep02_02_result_vi.csv')
data_vo = get_oscilloscpoe_result_tektronix('./data/ch06_ep02_02_result_vo.csv')

if data_vi != None and data_vo != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times_vi = data_vi['TIME']
    vins = data_vi['Sample CH1']
    times_vo = data_vo['TIME']
    vouts = data_vo['Sample CH1']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 10, 2)
    vouts = savgol_filter(vouts, 10, 2)

    # 첫 번째 첨두값을 기준으로 시간축을 동기화한다.
    vins_fn = interpolate.interp1d(times_vi, vins)
    vouts_fn = interpolate.interp1d(times_vo, vouts)
    start_time = max(times_vi[0], times_vo[0])
    end_time = min(times_vi[-1], times_vo[-1])
    times = np.arange(start_time, end_time, (end_time - start_time)/1000)
    vins = vins_fn(times)
    vouts = vouts_fn(times)
    vins_tmp = savgol_filter(vins, 50, 2)
    vouts_tmp = savgol_filter(vouts, 50, 2)
    vi_pi = find_first_peak_index(times, vins_tmp)
    vo_pi = find_first_peak_index(times, vouts_tmp)
    index_diff = vi_pi - vo_pi
    if index_diff > 0:
        times = times[index_diff:]
        vins = vins[index_diff:]
        vouts = vouts[:-index_diff]
    elif index_diff < 0:
        times = times[:index_diff]
        vins = vins[:index_diff:]
        vouts = vouts[-index_diff:]

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.xlim((0, end_time))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT)', 1)
    plt.show()

    vi_pi = find_first_peak_index(times, vins)
    vo_pi = find_first_peak_index(times, vouts)
    V_diff_V = vins[vi_pi] - vouts[vo_pi]
    print_value("V_diff_V", V_diff_V)

    # 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
    plt.xlabel('V_IN_V')
    plt.ylabel('V_OUT_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    m = int(len(vins) / 10)
    draw_plot(vins[m:-m], vouts[m:-m], "XY", 0)
    plt.show()


* 고찰:
    + Vi가 ?? 보다 클 때는 ??이 Vo로 출력된다.
    + 결과적으로 ??정류가 이루어진다.

### 실험02-03 트랜스포머로 실험

#### 실험 방법

* Vi: 트랜스포머 (AC 220 V -> 약 25 Vpp)
* Vi와 Vo를 각각 측정한다. (동시에 측정하면 Vi-가 GND와 연결되어 정상동작하지 않음)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep02_03_result_circuit.jpg", width=1000)

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep02_03_result_vi.png", width=600)
display_image("./data/ch06_ep02_03_result_vo.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data_vi = get_oscilloscpoe_result_tektronix('./data/ch06_ep02_03_result_vi.csv')
data_vo = get_oscilloscpoe_result_tektronix('./data/ch06_ep02_03_result_vo.csv')

if data_vi != None and data_vo != None:
    times_vi = data_vi['TIME']
    vins = data_vi['Sample CH1']
    times_vo = data_vo['TIME']
    vouts = data_vo['Sample CH1']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 10, 2)
    vouts = savgol_filter(vouts, 10, 2)

    # 첫 번째 첨두값을 기준으로 시간축을 동기화한다.
    vins_fn = interpolate.interp1d(times_vi, vins)
    vouts_fn = interpolate.interp1d(times_vo, vouts)

    start_time = max(times_vi[0], times_vo[0])
    end_time = min(times_vi[-1], times_vo[-1])
    times = np.arange(start_time, end_time, (end_time - start_time)/1000)
    vins = vins_fn(times)
    vouts = vouts_fn(times)
    vins_tmp = savgol_filter(vins, 50, 2)
    vouts_tmp = savgol_filter(vouts, 50, 2)
    vi_pi = find_first_peak_index(times, vins_tmp)
    vo_pi = find_first_peak_index(times, vouts_tmp)
    index_diff = vi_pi - vo_pi
    if index_diff > 0:
        times = times[index_diff:]
        vins = vins[index_diff:]
        vouts = vouts[:-index_diff]
    elif index_diff < 0:
        times = times[:index_diff]
        vins = vins[:index_diff:]
        vouts = vouts[-index_diff:]

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.xlim((0, end_time))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT)', 1)
    plt.show()

    vi_pi = find_first_peak_index(times, vins)
    vo_pi = find_first_peak_index(times, vouts)
    V_diff_V = vins[vi_pi] - vouts[vo_pi]
    print_value("V_diff_V", V_diff_V)

    # 입력 전압을 x축, 출력 전압을 y축으로 하는 그래프를 그린다.
    plt.xlabel('V_IN_V')
    plt.ylabel('V_OUT_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    m = int(len(vins) / 10)
    draw_plot(vins[m:-m], vouts[m:-m], "XY", 0)
    plt.show()


* 고찰:
    + 실험02-02 유사하게 ??정류가 이루어진다.

*******************************************************************************
## 실험03 평활 회로

### 실험03-01 회로의 구성

#### 실험 방법

* 다이오드: 1N4004
* 커패시터: 10 uF
* RL: 5.1 kOhm, 1 kOhm

In [ ]:
display_image("./images/ch06_ep03_01_circuit.png", width=600)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep03_01_result_circuit.jpg", width=1000)

### 실험03-02 파형발생기로 실험

#### 실험 방법

* Vi: 정현파, 주파수 60 Hz, 크기 12 Vpp

#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep03_02_result_5_1.png", width=600)
display_image("./data/ch06_ep03_02_result_1_0.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data_5_1 = get_oscilloscpoe_result_tektronix('./data/ch06_ep03_02_result_5_1.csv')
data_1_0 = get_oscilloscpoe_result_tektronix('./data/ch06_ep03_02_result_1_0.csv')

if data_5_1 != None and data_1_0 != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data_5_1['TIME']
    vins = data_5_1['Sample CH1']
    vouts_5_1 = data_5_1['Sample CH2']
    vouts_1_0 = data_1_0['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts_5_1 = savgol_filter(vouts_5_1, 10, 2)
    vouts_1_0 = savgol_filter(vouts_1_0, 10, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts_5_1, 'V(OUT, RL=5.1 kOhm)', 1)
    draw_plot(times, vouts_1_0, 'V(OUT, RL=1.0 kOhm)', 2)
    plt.show()


* 고찰:
    + Vo의 ??가 감소한다.
    + 부하저항이 더 ??면 Vo의 ??가 더 ??한다.

### 실험03-03 트랜스포머로 실험

#### 실험 방법

* Vi: 트랜스포머 (AC 220 V -> 약 25 Vpp)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep03_03_result_circuit.jpg", width=1000)

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep03_03_result_5_1.png", width=600)
display_image("./data/ch06_ep03_03_result_1_0.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data_5_1 = get_oscilloscpoe_result_tektronix('./data/ch06_ep03_03_result_5_1.csv')
data_1_0 = get_oscilloscpoe_result_tektronix('./data/ch06_ep03_03_result_1_0.csv')

if data_5_1 != None and data_1_0 != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data_5_1['TIME']
    vins = data_5_1['Sample CH1']
    vouts_5_1 = data_5_1['Sample CH2']
    vouts_1_0 = data_1_0['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts_5_1 = savgol_filter(vouts_5_1, 100, 2)
    vouts_1_0 = savgol_filter(vouts_1_0, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts_5_1, 'V(OUT, RL=5.1 kOhm)', 1)
    draw_plot(times, vouts_1_0, 'V(OUT, RL=1.0 kOhm)', 2)
    plt.show()


* 고찰:
    + 실험03-02 유사하게 Vo의 ??가 감소한다.

*******************************************************************************
## 실험04 정전압 회로

### 실험04-01 회로의 구성

#### 실험 방법

* 레귤레이터: LM7805
* 다이오드: 1N4004
* 커패시터: 100 uF, 10 uF
* RL: 무한대(open), 1 kOhm, 3 kOhm

In [ ]:
display_image("./images/ch06_ep04_01_circuit.png", width=600)

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep04_01_result_circuit.jpg", width=1000)

### 실험04-02 파형발생기로 실험

#### 실험 방법

* Vi 소스: 정현파, 주파수 60 Hz, 크기 20 Vpp ~ 18 Vpp

* 채널 1과 2의 Volts/div를 모두 1V로 설정한다.
* 채널 1과 2의 0V 기준점을 모두 화면의 가장 밑으로 보낸다.
* 트리거 채널을 1로, 레벨을 채널 1의 최고점과 최저점 중간으로 설정한다.


#### 실험 결과

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep04_02_result_infinte_20vpp.png", width=600)
display_image("./data/ch06_ep04_02_result_1kohm_20vpp.png", width=600)
display_image("./data/ch06_ep04_02_result_3kohm_20vpp.png", width=600)
display_image("./data/ch06_ep04_02_result_3kohm_18vpp.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep04_02_result_infinte_20vpp.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts = savgol_filter(vouts, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.ylim((-1, 12))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT, RL=infinte, 20 Vpp)', 1)
    plt.show()


* 고찰:
    + ??이 출력된다.
    + 하지만, 실제 정류기 사용 환경에서는 ?? 때문에 의미가 ??.

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep04_02_result_1kohm_20vpp.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts = savgol_filter(vouts, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.ylim((-1, 12))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT, RL=1 kOhm)', 1)
    plt.show()


* 고찰:
    + ??이 출력된다.

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep04_02_result_3kohm_20vpp.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts = savgol_filter(vouts, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.ylim((-1, 12))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT, RL=3 kOhm)', 1)
    plt.show()


* 고찰:
    + ??이 출력된다.
    + 하지만, 실제 정류기 사용 환경에서는 ?? 때문에 의미가 ??.

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep04_02_result_3kohm_18vpp.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts = savgol_filter(vouts, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.ylim((-1, 12))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT, RL=3 kOhm, 18 Vpp)', 1)
    plt.show()


* 고찰:
    + Dropout voltage가 ?? 때문에 ??이 출력된다. 

### 실험04-03 트랜스포머로 실험

#### 실험 방법

* Vi 소스: 트랜스포머 (AC 220 V -> 약 25 Vpp)
* RL: 1 kOhm

* 채널 1과 2의 Volts/div를 모두 2V로 설정한다.
* 채널 1과 2의 0V 기준점을 모두 화면의 가장 밑으로 보낸다.
* 트리거 채널을 1로, 레벨을 채널 1의 최고점과 최저점 중간으로 설정한다.

#### 실험 결과

##### 제작한 회로

In [ ]:
display_image("./data/ch06_ep04_03_result_circuit.jpg", width=1000)

##### 오실로스코프 화면

In [ ]:
display_image("./data/ch06_ep04_03_result.png", width=600)

#### 결과 분석

In [ ]:
# 실험 결과 파일을 읽는다.
data = get_oscilloscpoe_result_tektronix('./data/ch06_ep04_03_result.csv')

if data != None:
    # 실험 결과에서 시간, 입력 전압, 출력 전압을 가져온다.
    times = data['TIME']
    vins = data['Sample CH1']
    vouts = data['Sample CH2']

    # 필터를 적용해 잡음을 줄인다.
    vins = savgol_filter(vins, 100, 2)
    vouts = savgol_filter(vouts, 100, 2)

    # 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.
    plt.xlabel('TIME_s')
    plt.ylabel('V_V')
    plt.axhline(0, color='black')
    plt.axvline(0, color='black')
    plt.ylim((-1, 12))
    draw_plot(times, vins, 'V(IN)', 0)
    draw_plot(times, vouts, 'V(OUT, RL=1 kOhm)', 1)
    plt.show()


* 고찰:
    + Dropout voltage가 ?? 때문에 ?? 불구하고 ??이 출력된다.